This notebook was created by Skylar Miller and Ellis Hartley

# Assumptions

- Domestic company only interested in domestic gross revenue (for seasonality consistency)
- 


# Question 

Does the season of release affect the revenue of a movie?

# Backround 

We think that the summer months will perform the best because students are out of school and families will be looking for ways to entertain. In addition, with heat waves in full effect, in activities are preferred. 

### Null Hypothesis: There is no difference between summer month revenue and other month revenues. 
 
### Alternative Hypothesis: There is more summer month revenue than other month revenues.

# Data Exploration
- Opening all of the available data sources, we decided to go with The Numbers Dataset because it is the only one that contains the information that we are interested in. More info on this dataset can be found here: https://www.the-numbers.com/

In [1]:
import pandas as pd
df_bom = pd.read_csv('../../zippedData/bom.movie_gross.csv.gz')
df_bom.head(1)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010


In [2]:
df_tmbd = pd.read_csv('../../zippedData/tmdb.movies.csv.gz')
df_tmbd.head(1)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788


In [51]:
df_tn = pd.read_csv('../../zippedData/tn.movie_budgets.csv.gz')
df_tn.head(1)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"


In [4]:
df_rt_info = pd.read_csv('../../zippedData/rt.movie_info.tsv.gz', sep='\t')
df_rt_info.head(1)

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN


In [5]:
df_rt_reviews = pd.read_csv('../../zippedData/rt.reviews.tsv.gz', sep='\t', encoding='ansi')
df_rt_reviews.head(1)

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"


# Our analysis starts here:

In [7]:
df_tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


As shown above, this dataset consists of 5782 rows with no Null values. 

In [52]:
def clean_dollar(df, column):
    df[column] = df[column].str.strip('$')
    df[column] = df[column].str.replace(',','')
    df[column] = df[column].astype(int)
    return df[column]

clean_dollar(df_tn,'production_budget') 
clean_dollar(df_tn,'domestic_gross')
# clean_dollar(df_tn,'worldwide_gross')

0       760507625
1       241063875
2        42762350
3       459005868
4       620181382
          ...    
5777            0
5778        48482
5779         1338
5780            0
5781       181041
Name: domestic_gross, Length: 5782, dtype: int32

In [53]:
df_tn.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,"$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,"$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,"$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,"$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,"$1,316,721,747"


In [54]:
# Create a new row called 'seasons' that categorizes the dates columns by season. Seasons are as follows:

# summer = june 21 - sept 22 (06/21-09/22) # summer months = 6,7,8
# fall = sept 23 - dec 20 (09/23-12/20) # fall months = 9,10,11
# winter = dec 21 - mar 18 (12/21-03/18) # winter months = 12,1,2
# spring = mar 19 - jun 20 (03/19-06/20) # spring months = 3,4,5

summer = ('Jun', 'Jul', 'Aug')
fall = ('Sep', 'Oct', 'Nov')
winter = ('Dec', 'Jan', 'Feb')
spring = ('Mar', 'Apr', 'May')

def f(row):
    if row['release_date'].startswith(summer):
        val = 'Summer'
    elif row['release_date'].startswith(fall):
        val = 'Fall'
    elif row['release_date'].startswith(winter):
        val = 'Winter'
    else:
        val = 'Spring'
    return val

df_tn['seasons'] = df_tn.apply(f, axis=1)

In [55]:
df_tn.head(10)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,seasons
0,1,"Dec 18, 2009",Avatar,425000000,760507625,"$2,776,345,279",Winter
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,"$1,045,663,875",Spring
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,"$149,762,350",Summer
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,"$1,403,013,963",Spring
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,"$1,316,721,747",Winter
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000,936662225,"$2,053,311,220",Winter
6,7,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,"$2,048,134,200",Spring
7,8,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,300000000,309420425,"$963,420,425",Spring
8,9,"Nov 17, 2017",Justice League,300000000,229024295,"$655,945,209",Fall
9,10,"Nov 6, 2015",Spectre,300000000,200074175,"$879,620,923",Fall


In [56]:
df_tn.sort_values(by=['domestic_gross'], ascending=True).head(10)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,seasons
3231,32,"Dec 31, 2008",Ca$h,13700000,0,"$11,738,256",Winter
4762,63,"Dec 31, 2009",Chicago Overcoat,3000000,0,$0,Winter
4763,64,"Sep 13, 2011",Never Back Down 2: The Beatdown,3000000,0,$0,Fall
4764,65,"Dec 31, 2015",AWOL-72,3000000,0,$0,Winter
4194,95,"May 10, 2011",The Hit List,6000000,0,$0,Spring
4765,66,"Jan 15, 2013",30 Nights Of Paranormal Activity With The Devi...,3000000,0,$0,Winter
4766,67,"Feb 6, 2015",Enter the Dangerous Mind,3000000,0,$0,Winter
5207,8,"Dec 31, 2013",The Vatican Exorcisms,1100000,0,$0,Winter
5206,7,"Dec 31, 2010",The Dead Undead,1100000,0,$0,Winter
5205,6,"Apr 7, 2015",Silent Running,1100000,0,$0,Spring


In [57]:
df_tn.sort_values(by=['domestic_gross'], ascending=False).head(10)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,seasons
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000,936662225,"$2,053,311,220",Winter
0,1,"Dec 18, 2009",Avatar,425000000,760507625,"$2,776,345,279",Winter
41,42,"Feb 16, 2018",Black Panther,200000000,700059566,"$1,348,258,224",Winter
6,7,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,"$2,048,134,200",Spring
42,43,"Dec 19, 1997",Titanic,200000000,659363944,"$2,208,208,395",Winter
33,34,"Jun 12, 2015",Jurassic World,215000000,652270625,"$1,648,854,864",Summer
26,27,"May 4, 2012",The Avengers,225000000,623279547,"$1,517,935,897",Spring
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,"$1,316,721,747",Winter
43,44,"Jun 15, 2018",Incredibles 2,200000000,608581744,"$1,242,520,711",Summer
74,75,"Jul 18, 2008",The Dark Knight,185000000,533720947,"$1,001,996,207",Summer


It looks like there are a lot of movies with 0 domestic gross revenue. Since we are only interested in domestic, we are going to drop those movies from our dataset. Below we see there are 548 rows to drop.

In [59]:
df_zeros = df_tn[df_tn['domestic_gross']==0]
print(len(df_zeros))
df_zeros

548


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,seasons
194,95,"Dec 31, 2020",Moonfall,150000000,0,$0,Winter
479,80,"Dec 13, 2017",Bright,90000000,0,$0,Winter
480,81,"Dec 31, 2019",Army of the Dead,90000000,0,$0,Winter
535,36,"Feb 21, 2020",Call of the Wild,82000000,0,$0,Winter
617,18,"Dec 31, 2012",AstÃ©rix et ObÃ©lix: Au service de Sa MajestÃ©,77600000,0,"$60,680,125",Winter
...,...,...,...,...,...,...,...
5761,62,"Dec 31, 2014",Stories of Our Lives,15000,0,$0,Winter
5764,65,"Dec 31, 2007",Tin Can Man,12000,0,$0,Winter
5771,72,"May 19, 2015",Family Motocross,10000,0,$0,Spring
5777,78,"Dec 31, 2018",Red 11,7000,0,$0,Winter


In [60]:
df = df_tn.drop(df_tn[df_tn['domestic_gross']==0].index)

df.sort_values(by=['domestic_gross'], ascending=True).head(10)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,seasons
3731,32,"Dec 26, 2014",Barbecue,10000000,388,"$247,565",Winter
5770,71,"Aug 14, 2008",The Rise and Fall of Miss Thang,10000,401,$401,Summer
5518,19,"Oct 13, 2005",The Dark Hours,400000,423,$423,Fall
5769,70,"Apr 1, 1996",Bang,10000,527,$527,Spring
5466,67,"May 11, 2018",Higher Power,500000,528,$528,Spring
5027,28,"Jan 1, 1993",Ed and his Dead Mother,1800000,673,$673,Winter
5298,99,"Dec 13, 2002",The Jimmy Show,1000000,703,$703,Winter
4422,23,"Oct 19, 2007",Out of the Blue,4600000,728,"$731,767",Fall
4081,82,"May 21, 2010",Perrierâs Bounty,6600000,828,$828,Spring
5297,98,"Apr 21, 2006",In Her Line of Fire,1000000,884,$884,Spring


We probably want to do something with these low grossing films. A quick google search on "Ed and his Dead Mother" shows a few well-known actors so these numbers are probably incorrect. 

In [62]:
df.describe()

,id,production_budget,domestic_gross
count,5234.000000,5.234000e+03,5.234000e+03
mean,50.346580,3.403348e+07,4.625747e+07
std,28.848667,4.296048e+07,7.029651e+07
min,1.000000,1.100000e+03,3.880000e+02
25%,25.000000,6.500000e+06,4.289718e+06
50%,50.000000,2.000000e+07,2.198422e+07
75%,75.000000,4.500000e+07,5.756598e+07
max,100.000000,4.250000e+08,9.366622e+08
